In [1]:
import pandas as pd
import numpy as np
import os
import random
from sklearn.model_selection import train_test_split
import shutil
import gdown

connect google drive and move to folder

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! ls /content/drive/MyDrive/CodingTask  ## find this path in your drive

pm-back  pm-full


In [4]:
!cp -r /content/drive/MyDrive/CodingTask /content/

In [5]:
import os
import shutil
from sklearn.model_selection import train_test_split

In [6]:
base_dir = "/content/CodingTask/"
front_dir = os.path.join(base_dir, "pm-full")
back_dir = os.path.join(base_dir, "pm-back")

In [7]:
split_base_dir = "/content/CodingTaskSplits/"
os.makedirs(split_base_dir, exist_ok=True)

splits = ["train", "val", "test"]
classes = ["pm-full", "pm-back"]


In [8]:
for split in splits:
    for cls in classes:
        os.makedirs(os.path.join(split_base_dir, split, cls), exist_ok=True)

In [9]:
def split_and_move_files(class_dir, dest_dirs, split_ratios):
    files = os.listdir(class_dir)
    files = [os.path.join(class_dir, f) for f in files if os.path.isfile(os.path.join(class_dir, f))]

    train_files, temp_files = train_test_split(files, test_size=(split_ratios[1] + split_ratios[2]), random_state=42)
    val_files, test_files = train_test_split(temp_files, test_size=split_ratios[2] / (split_ratios[1] + split_ratios[2]), random_state=42)

    # Move files to respective directories
    for f in train_files:
        shutil.copy(f, os.path.join(dest_dirs['train'], os.path.basename(f)))
    for f in val_files:
        shutil.copy(f, os.path.join(dest_dirs['val'], os.path.basename(f)))
    for f in test_files:
        shutil.copy(f, os.path.join(dest_dirs['test'], os.path.basename(f)))

# Perform split for 'front' and 'back'
split_ratios = [0.7, 0.15, 0.15]  # Train: 70%, Val: 15%, Test: 15%
for cls in classes:
    class_dir = os.path.join(base_dir, cls)
    dest_dirs = {split: os.path.join(split_base_dir, split, cls) for split in splits}
    split_and_move_files(class_dir, dest_dirs, split_ratios)

In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from sklearn.metrics import classification_report

# Paths to the data splits
base_dir = "/content/CodingTaskSplits"
train_dir = f"{base_dir}/train"
val_dir = f"{base_dir}/val"
test_dir = f"{base_dir}/test"

# Image parameters
IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZE = 32

In [11]:
train_datagen = ImageDataGenerator(rescale=1.0/255)
val_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 77 images belonging to 2 classes.
Found 18 images belonging to 2 classes.
Found 19 images belonging to 2 classes.


In [19]:
base_model = MobileNetV2(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
                         include_top=False,  # Remove the original classification head
                         weights='imagenet')

# Freeze the base model
base_model.trainable = False

# Add a new classification head
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

In [20]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [21]:
EPOCHS = 10
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS
)

# Unfreeze some layers of the base model and fine-tune
base_model.trainable = True

# Fine-tuning with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.5656 - loss: 0.8352 - val_accuracy: 0.8333 - val_loss: 0.3903
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 125ms/step - accuracy: 0.8336 - loss: 0.3816 - val_accuracy: 0.8889 - val_loss: 0.3164
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 128ms/step - accuracy: 0.9076 - loss: 0.2070 - val_accuracy: 0.8333 - val_loss: 0.3101
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 122ms/step - accuracy: 0.9024 - loss: 0.1908 - val_accuracy: 0.8889 - val_loss: 0.2541
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 171ms/step - accuracy: 0.9545 - loss: 0.1428 - val_accuracy: 0.9444 - val_loss: 0.2681
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 173ms/step - accuracy: 0.9363 - loss: 0.1212 - val_accuracy: 0.8889 - val_loss: 0.2423
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 173ms/step - accuracy: 0.9815 - loss: 0.0772 - val_accuracy: 0.8889 - val_loss: 0.2249
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 339ms/step - accuracy: 0.9750 - loss: 0.0636 - val_accuracy: 0.8889 - val_loss: 0

In [22]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.8421 - loss: 0.1992
Test Accuracy: 0.8421052694320679


In [23]:
fine_tune_epochs = 10
base_model.trainable = False
history_fine_tune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=fine_tune_epochs
)

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 1.0000 - loss: 0.0380 - val_accuracy: 0.8889 - val_loss: 0.2352
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 128ms/step - accuracy: 0.9880 - loss: 0.0366 - val_accuracy: 0.8889 - val_loss: 0.2354
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 133ms/step - accuracy: 1.0000 - loss: 0.0172 - val_accuracy: 0.8889 - val_loss: 0.2358
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 120ms/step - accuracy: 1.0000 - loss: 0.0236 - val_accuracy: 0.8889 - val_loss: 0.2362
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 128ms/step - accuracy: 1.0000 - loss: 0.0262 - val_accuracy: 0.8889 - val_loss: 0.2366
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step - accuracy: 1.0000 - loss: 0.0303 - val_accuracy: 0.8889 - val_loss: 0.2369
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 175ms/step - accuracy: 1.0000 - loss: 0.0321 - val_accuracy: 0.8889 - val_loss: 0.2371
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 174ms/step - accuracy: 1.0000 - loss: 0.0412 - val_accuracy: 0.8889 - val_loss: 0

In [24]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step - accuracy: 0.8947 - loss: 0.2018
Test Accuracy: 0.8947368264198303


In [32]:
test_generator.reset()
df = pd.DataFrame(
)
df['y_pred'] = model.predict(test_generator).flatten()
df['y_true'] = test_generator.classes
df['label'] = (df['y_pred'] > 0.6).astype(int)
print(classification_report(df["y_true"], df['label'], target_names=test_generator.class_indices.keys()))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step
              precision    recall  f1-score   support

     pm-back       0.90      0.90      0.90        10
     pm-full       0.89      0.89      0.89         9

    accuracy                           0.89        19
   macro avg       0.89      0.89      0.89        19
weighted avg       0.89      0.89      0.89        19



In [34]:
model.save("/content/front_back_classifier_mobilenetv2.h5")